# AGENTIC BLOGGER

Agentic Blog Post Writing Workflow using LangGraph and GPT4.1.

This script defines a multi-agent graph using LangGraph where each node represents a step in the blog post writing process: planning, research, writing, reflection, critique, and revision.

## Import Modules

In [2]:
#import environment file
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

# Modules for structuring text
from typing import Annotated
from typing_extensions import TypedDict
from typing import List, Union, Callable
from pydantic import BaseModel, Field

# LangGraph modules for defining graphs
from langgraph.graph import MessagesState, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

# Modeules for Messages
from langchain_core.messages import AIMessage, HumanMessage

# Module for setting up OpenAI
from langchain_openai import ChatOpenAI

# Modules for creating ReAct agents with Supervisor architecture
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent

# Import modules for chatbot diagram
from IPython.display import Image, display

# Import the modules for saving memory
from langgraph.checkpoint.memory import MemorySaver

# Initialize LLMs

### GPT

In [3]:
# Load API Key
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

In [18]:
gpt = ChatOpenAI(
    model_name="gpt-4.1-mini-2025-04-14",
    temperature=1
)

# Define workflow functions

## Create specialized agents

In [16]:
planning_agent = create_react_agent(
    model=gpt,
    tools=[],
    name="planning_expert",
    prompt="""
    You are a world-class, meticulous blog post planner.
    Create a high-level plan (outline) with key details for a blog post based on the user's prompt.
    The plan should include key sections, main points for each section, and suggest a target audience and tone.
    """
)

research_agent = create_react_agent(
    model=gpt,
    tools=[],
    name="research_agent",
    prompt="""
    You are a world-class, meticulous research assistant with access to google web search.
    You are a tasked with providing information that can be used for the blog post's content.
    Generate a list of search queries that will gather relevant information.
    Only generate 3-5 queries max.
    """
)

writer_agent = create_react_agent(
    model=gpt,
    tools=[],
    name="writing_agent",
    prompt="""
    You are a world-class assistant that writes blog post for LinkedIn that grab people's attention
    and their curiosity. Generate a viral blog post based on the user's request and the initial outline.
    Write in the style of AI by Design-
    the mission of AI by Design is to deliver weekly insights for Product Managers who want to launch and scale 0→1 products with AI.
    
    Here are 3 examples of their previous blog posts; 
    you have access to web search when referencing the URLs:
    
    1. https://aibydesign.io/blog/openai-agentic-whitepaper-missed-the-point
    2. https://aibydesign.io/blog/mastering-ai-product-strategy
    3. https://aibydesign.io/blog/embracing-failure
    """
)

critique_agent = create_react_agent(
    model=gpt,
    tools=[],
    name="critique_agent",
    prompt="""
    You are a world-class, thoughtful reviewer. 
    Review the blog post draft written about the user's prompt and based on the plan reflect on its strengths and weaknesses. 
    Consider: Does it follow the plan? Is it coherent? Is the tone right?
    Does it address the initial idea effectively?
    Is the research integrated well? 
    Provide specific critique and recommendations for improvement.
    Focus on clarity, engagement, length, style, and structure.
    List the key areas to revise.
    """
)

revision_agent = create_react_agent(
    model=gpt,
    tools=[],
    name="revision_agent",
    prompt="""
    You are an expert editor.
    Revise the following blog post draft about the user's prompt based *strictly* on the provided reflection and critique points.
    Address *all* points mentioned.
    Produce the polished, final version of the blog post.
    
    """
)

# Create supervisor workflow
workflow = create_supervisor(
    [planning_agent, research_agent,
    writer_agent, critique_agent, revision_agent],
    model=gpt,
    prompt=("""
        You are a word-class team supervisor managing a planning_agent (creates a high-level outline with key details for a blog post based on the user's prompt), 
        a research_agent (conducts research that can be used for the blog post's content),
        a writing_agent (writes viral blog post drafts for LinkedIn that grab people's attention and curiosity), 
        a critique_agent (review the draft written about the user's prompt and provides specific critique and recommendations for improvement),
        a revision_agent (an expert editor that revises the based on the critique to produce the final version of the blog post)
        
        You are tasked with delegating to and managing the team to write viral blog posts for LinkedIn audience that capture the audience's curiosity and attention.

        The handoffs should always follow this sequential process:
        1. planning_agent 
        2. research_agent
        3. writing_agent 
        4. critique_agent 
        5. revision_agent

        *Very Important* always pass the relevant context from the prior agent to the next agent, including outline, research, draft, and critique.
        Finally, output the final draft.
    """),
    output_mode="last_message"
    # output_mode="full_history"
)

memory = MemorySaver()
app = workflow.compile(
    checkpointer=memory
)

In [9]:
# display(Image(app.get_graph().draw_mermaid_png()))

## Follow-up questions with multiple tools

In [17]:
config = {"configurable": {"thread_id": "1"}}

# Print the user query first for every interaction
def user_agent_multiturn(queries):
    for query in queries:
        print(f"User: {query}")

        # Stream through messages corresponding to queries, excluding metadata
        print("Agent: " + "".join(msg.content for msg, metadata in app.stream(
            {"messages": [HumanMessage(content=query)]}, config, stream_mode="messages")
            if msg.content and not isinstance(msg, HumanMessage)) + "\n")

queries = ["what is this article about? https://blog.langchain.dev/how-to-think-about-agent-frameworks/", 
           "now craft it into a viral blog post that will grab reader's attention and occupy that curiosity.  \
           Keep in mind that this was a hot take 🔥 from Harrison Chase, the co-Founder of Langchain in response to OpenAI's agents SDK whitepaper"]
user_agent_multiturn(queries)

User: what is this article about? https://blog.langchain.dev/how-to-think-about-agent-frameworks/
Agent: Here are some search queries to gather relevant information about the article "How to Think About Agent Frameworks" from the LangChain blog:

1. "How to think about agent frameworks LangChain blog summary"
2. "Agent frameworks in LangChain explained"
3. "Key concepts in LangChain agent frameworks"
4. "Overview of agent frameworks in AI from LangChain"
5. "LangChain blog article on agent frameworks main points"

Would you like me to summarize the article based on its content or fetch key highlights from these queries?Transferring back to supervisorSuccessfully transferred back to supervisorThe article "How to Think About Agent Frameworks" on the LangChain blog discusses the conceptual understanding and design considerations surrounding agent frameworks in AI development. It explains what agent frameworks are, their importance in building intelligent systems, and how they help develop

# Next Steps
- Short & Long Memory
- Human in the loop
- Experiment with thinking budget
- Define workflow
- Replit UI